In [9]:
import numpy as np
import nnfs

from nnfs.datasets import spiral_data
nnfs.init()

In [10]:
inputs = np.array([[1, 2, 3, 2.5],
                   [2, 5, -1, 2],
                   [-1.5, 2.7, 3.3,-.8]])

weights = np.array([[.2, .8, -.5, 1],
                    [.5, -.91, .26, -.5],
                    [-.26, -.27, .17, .87]])

biases = [2, 3, .5]

layer1_outputs = np.dot(inputs, weights.T) + biases

In [11]:
weights2 = np.array([[.1, -.14, .5],
                    [-.5, .12, -.33],
                    [-.44, .73, -.13]])

biases2 = [-1,2,-.5]

layer2_outputs = np.dot(layer1_outputs, weights2.T) + biases2
layer2_outputs

array([[ 0.50310004, -1.04184985, -2.03874993],
       [ 0.24339998, -2.73320007, -5.76329994],
       [-0.99314   ,  1.41254002, -0.35655001]])

In [12]:
class Layer_Dense:
    
    def __init__(self, n_inputs, n_neurons):
        self.weights = .1 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

class Activation_ReLU:
    
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

class Activation_Softmax:
    
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))

        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

class Loss:
            
    def calculate(self, output, y):

        sample_losses = self.forward(output, y)
        
        return np.mean(sample_losses)

class Loss_CatCrossEntropy(Loss):

    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)

        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        else:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis = 1)

        return -np.log(correct_confidences)

In [13]:
X, y = spiral_data(samples=100, classes=3)

dense1 = Layer_Dense(2,3)
activation1 = Activation_ReLU()

dense2 = Layer_Dense(3,3)
activation2 = Activation_Softmax()


dense1.forward(X)
activation1.forward(dense1.output)

dense2.forward(activation1.output)
activation2.forward(dense2.output)

activation2.output[:5]

array([[0.33333334, 0.33333334, 0.33333334],
       [0.33331734, 0.3333183 , 0.33336434],
       [0.3332888 , 0.33329153, 0.33341965],
       [0.33325943, 0.33326396, 0.33347666],
       [0.33323312, 0.33323926, 0.33352762]], dtype=float32)

In [14]:
loss_fucntion = Loss_CatCrossEntropy()
loss = loss_fucntion.calculate(activation2.output, y)
print(loss)

1.098445
